# 第5章: 大規模言語モデル

この章では, 大規模言語モデル (LLM; Large Language Model) の利用し, 様々なタスクに取り組む. 大規模言語モデルをプログラムからAPI経由で呼び出すことを想定しており, そのAPIの利用で費用が発生する可能性があることに留意せよ.

```{warning}
本章は, `code-cell` ではなく, Markdown のコードブロック内にコードを記述しているため, Google Colab上で直接実行できません.
```

In [1]:
# !pip install vllm

## 40. Zero-Shot推論

以下の問題の解答を作成せよ. ただし, 解答生成はzero-shot推論とせよ.

```
9世紀に活躍した人物に関係するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。
イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。
ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。
```

出典: [令和5年度第1回高等学校卒業程度認定試験問題](https://www.mext.go.jp/a_menu/koutou/shiken/kakomon/1411255_00010.htm) [日本史AB](https://www.mext.go.jp/content/20240523-mxt_syogai02-mext_000031286_03nihonshi.pdf) 問題 日本史B 1 問3

```python
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

model_name = "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = LLM(
    model=model_name,
    tensor_parallel_size=1,
    dtype="float16"
)

sampling_params = SamplingParams(
    temperature=0.6, top_p=0.9, max_tokens=512, stop="<|eot_id|>"
)

system_prompt = {
    "role": "system", 
    "content": "あなたは日本の歴史学者です。"
}

user_prompt = """9世紀に活躍した人物に関係するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。
イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。
ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。"""

message = [
    system_prompt,
    {
        "role": "user",
        "content": user_prompt
    }
]

prompt = tokenizer.apply_chat_template(
    message, tokenize=False, add_generation_prompt=True
)

outputs = llm.generate(prompt, sampling_params)
result = outputs[0].outputs[0].text.strip()

print("解答:", result)
```

```bash
解答: 正しい年代の古い順に並べると次のようになります。

1. **イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。** (809年)
2. **ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。** (842年)
3. **ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。** (903年)
```

## 41. Few-Shot推論

以下の問題と解答を与え, 問題40で示した質問の解答をfew-shot推論（この場合は4-shot推論）で生成せよ.

```
日本の近代化に関連するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　府知事・県令からなる地方官会議が設置された。
イ　廃藩置県が実施され，中央から府知事・県令が派遣される体制になった。
ウ　すべての藩主が，天皇に領地と領民を返還した。

解答: ウ→イ→ア
```

出典: [令和5年度第1回高等学校卒業程度認定試験問題](https://www.mext.go.jp/a_menu/koutou/shiken/kakomon/1411255_00010.htm) [日本史AB 問題](https://www.mext.go.jp/content/20240523-mxt_syogai02-mext_000031286_03nihonshi.pdf) 日本史A 1 問8

```
江戸幕府の北方での対外的な緊張について述べた次の文ア～ウを年代の古い順に正しく並べよ。

ア　レザノフが長崎に来航したが，幕府が冷淡な対応をしたため，ロシア船が樺太や択捉島を攻撃した。
イ　ゴローウニンが国後島に上陸し，幕府の役人に捕らえられ抑留された。
ウ　ラクスマンが根室に来航し，漂流民を届けるとともに通商を求めた。

解答: ウ→ア→イ
```

出典: [令和5年度第1回高等学校卒業程度認定試験問題](https://www.mext.go.jp/a_menu/koutou/shiken/kakomon/1411255_00010.htm) [日本史AB 問題](https://www.mext.go.jp/content/20240523-mxt_syogai02-mext_000031286_03nihonshi.pdf) 日本史B 3 問3

```
中居屋重兵衛の生涯の期間におこったできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。

ア　アヘン戦争がおこり，清がイギリスに敗北した。
イ　異国船打払令が出され，外国船を撃退することが命じられた。
ウ　桜田門外の変がおこり，大老の井伊直弼が暗殺された。

解答: イ→ア→ウ
```

出典: [令和4年度第1回高等学校卒業程度認定試験問題](https://www.mext.go.jp/a_menu/koutou/shiken/kakomon/1411255_00007.htm) [日本史 問題](https://www.mext.go.jp/content/20240513-mxt_syogai02-mext_00002452_03nihonshi.pdf) 日本史A 1 問1

```
加藤高明が外務大臣として提言を行ってから、内閣総理大臣となり演説を行うまでの時期のできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。

ア　朝鮮半島において，独立を求める大衆運動である三・一独立運動が展開された。
イ　関東大震災後の混乱のなかで，朝鮮人や中国人に対する殺傷事件がおきた。
ウ　日本政府が，袁世凱政府に対して二十一カ条の要求を突き付けた。

解答: ウ→ア→イ
```

出典: [令和4年度第1回高等学校卒業程度認定試験問題](https://www.mext.go.jp/a_menu/koutou/shiken/kakomon/1411255_00007.htm) [日本史 問題](https://www.mext.go.jp/content/20240513-mxt_syogai02-mext_00002452_03nihonshi.pdf) 日本史A 2 問4

```python
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

model_name = "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = LLM(
    model=model_name,
    tensor_parallel_size=1,
    dtype="float16"
)

sampling_params = SamplingParams(
    temperature=0.6, top_p=0.9, max_tokens=512, stop="<|eot_id|>"
)

system_prompt = {
    "role": "system", 
    "content": "あなたは日本の歴史学者です。"
}

fewshot_prompt = [
    {
        "role": "user",
        "content": """日本の近代化に関連するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　府知事・県令からなる地方官会議が設置された。
イ　廃藩置県が実施され，中央から府知事・県令が派遣される体制になった。
ウ　すべての藩主が，天皇に領地と領民を返還した。"""
    },
    {
        "role": "assistant",
        "content": "ウ→イ→ア"
    },
    {
        "role": "user",
        "content": """江戸幕府の北方での対外的な緊張について述べた次の文ア～ウを年代の古い順に正しく並べよ。

ア　レザノフが長崎に来航したが，幕府が冷淡な対応をしたため，ロシア船が樺太や択捉島を攻撃した。
イ　ゴローウニンが国後島に上陸し，幕府の役人に捕らえられ抑留された。
ウ　ラクスマンが根室に来航し，漂流民を届けるとともに通商を求めた。"""
    },
    {
        "role": "assistant",
        "content": "ウ→ア→イ"
    },
    {
        "role": "user",
        "content": """中居屋重兵衛の生涯の期間におこったできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。

ア　アヘン戦争がおこり，清がイギリスに敗北した。
イ　異国船打払令が出され，外国船を撃退することが命じられた。
ウ　桜田門外の変がおこり，大老の井伊直弼が暗殺された。"""
    },
    {
        "role": "assistant",
        "content": "イ→ア→ウ"
    },
    {
        "role": "user",
        "content": """加藤高明が外務大臣として提言を行ってから、内閣総理大臣となり演説を行うまでの時期のできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。

ア　朝鮮半島において，独立を求める大衆運動である三・一独立運動が展開された。
イ　関東大震災後の混乱のなかで，朝鮮人や中国人に対する殺傷事件がおきた。
ウ　日本政府が，袁世凱政府に対して二十一カ条の要求を突き付けた。"""
    },
    {
        "role": "assistant",
        "content": "ウ→ア→イ"
    }
]

user_prompt = """9世紀に活躍した人物に関係するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。
イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。
ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。"""

message = [
    system_prompt,
    *fewshot_prompt,
    {
        "role": "user",
        "content": user_prompt
    }
]

prompt = tokenizer.apply_chat_template(
    message, tokenize=False, add_generation_prompt=True
)

outputs = llm.generate(prompt, sampling_params)
result = outputs[0].outputs[0].text.strip()

print("解答:", result)
```

```bash
解答: イ→ウ→ア
```

## 42. 多肢選択問題の正解率

[JMMLU](https://github.com/nlp-waseda/JMMLU) のいずれかの科目を大規模言語モデルに解答させ, その正解率を求めよ.

In [2]:
# !git clone https://github.com/nlp-waseda/JMMLU.git

```python
import pandas as pd
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

df = pd.read_csv("JMMLU/JMMLU/abstract_algebra.csv", header=None)
df.columns = ["question", "selA", "selB", "selC", "selD", "answer"]

model_name = "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = LLM(
    model=model_name,
    tensor_parallel_size=1,
    dtype="float16", 
)

sampling_params = SamplingParams(
    temperature=0.6, top_p=0.9, max_tokens=512, stop="<|eot_id|>"
)

system_prompt = {
    "role": "system", 
    "content": "あなたは優れた抽象代数学の専門家です。以下の質問に対して所与の選択肢から正しい答えを選んでください。そのため、解答として許容される文字は「A」「B」「C」「D」のみです。解答は選択肢のいずれか1つを選ぶ形で行ってください。"
}

valid_answers = ['A', 'B', 'C', 'D']
predictions = []
invalid_cnt = 0

for index, row in df.iterrows():
    question_text = row["question"] + "\n" + f"A: {row['selA']} B: {row['selB']} C: {row['selC']} D: {row['selD']}"
    
    user_prompt = {
        "role": "user",
        "content": question_text
    }

    message = [system_prompt, user_prompt]
    
    prompt = tokenizer.apply_chat_template(
        message, tokenize=False, add_generation_prompt=True
    )

    answer = llm.generate(prompt, sampling_params)
    answer = answer[0].outputs[0].text.strip()

    if answer not in valid_answers:
        clarification_prompt = [
            system_prompt,
            user_prompt,
            {"role": "assistant", "content": answer},
            {
                "role": "user",
                "content": (
                    f"あなたの先ほどの回答「{answer}」はラベル (A/B/C/D) のいずれでもありません。"
                    "これは選択肢のどれに該当するか、ラベル (A/B/C/D) のみで1文字で答えてください。"
                )
            },
        ]

        clarification_text = tokenizer.apply_chat_template(
            clarification_prompt, tokenize=False, add_generation_prompt=True
        )

        clarification_answers = llm.generate(clarification_text, sampling_params)
        clarified_answer = clarification_answers[0].outputs[0].text.strip().upper()

        if clarified_answer in valid_answers:
            answer = clarified_answer
        else:
            answer = "Invalid"
            invalid_cnt += 1

    predictions.append(answer)
    
valid_indices = [i for i, p in enumerate(predictions) if p in valid_answers]
correct_predictions = [
    predictions[i] == df.loc[i, 'answer']
    for i in valid_indices
]

accuracy = sum(correct_predictions) / len(correct_predictions) if correct_predictions else 0.0
print(f"\nAccurasy: {accuracy * 100:.2f}% (Number of invalid predictions: {invalid_cnt})")
```

```bash
Accurasy: 31.96% (Number of invalid predictions: 2)
```

## 43. 応答のバイアス

問題42において, 実験設定を変化させると正解率が変化するかどうかを調べよ. 実験設定の例としては, 大規模言語モデルの温度パラメータ, プロンプト, 多肢選択肢の順番, 多肢選択肢の記号などが考えられる.

```python
import pandas as pd
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

df = pd.read_csv("JMMLU/JMMLU/abstract_algebra.csv", header=None)
df.columns = ["question", "selA", "selB", "selC", "selD", "answer"]

model_name = "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = LLM(
    model=model_name,
    tensor_parallel_size=1,
    dtype="float16", 
)

sampling_params = SamplingParams(
    temperature=0.6, top_p=0.9, max_tokens=512, stop="<|eot_id|>"
)

system_prompt = {
    "role": "system", 
    "content": "あなたは優れた抽象代数学の専門家です。以下の質問に対して所与の選択肢から正しい答えを選んでください。そのため、解答として許容される文字は「A」「B」「C」「D」のみです。解答は選択肢のいずれか1つを選ぶ形で行ってください。"
}

valid_answers = ['A', 'B', 'C', 'D']
predictions = []
invalid_cnt = 0

for index, row in df.iterrows():
    question_text = {
        "question": row["question"],
        "options": {
            "A": row["selA"],
            "B": row["selB"],
            "C": row["selC"],
            "D": row["selD"]
            }
        }
    
    user_prompt = {
        "role": "user",
        "content": question_text
    }

    message = [system_prompt, user_prompt]
    
    prompt = tokenizer.apply_chat_template(
        message, tokenize=False, add_generation_prompt=True
    )

    answer = llm.generate(prompt, sampling_params)
    answer = answer[0].outputs[0].text.strip()

    if answer not in valid_answers:
        clarification_prompt = [
            system_prompt,
            user_prompt,
            {"role": "assistant", "content": answer},
            {
                "role": "user",
                "content": (
                    f"あなたの先ほどの解答「{answer}」はラベル (A/B/C/D) のいずれでもありません。"
                    "これは選択肢のどれに該当するか、ラベル (A/B/C/D) のみで1文字で答えてください。"
                )
            },
        ]

        clarification_text = tokenizer.apply_chat_template(
            clarification_prompt, tokenize=False, add_generation_prompt=True
        )

        clarification_answers = llm.generate(clarification_text, sampling_params)
        clarified_answer = clarification_answers[0].outputs[0].text.strip().upper()

        if clarified_answer in valid_answers:
            answer = clarified_answer
        else:
            answer = "Invalid"
            invalid_cnt += 1

    predictions.append(answer)
    
valid_indices = [i for i, p in enumerate(predictions) if p in valid_answers]
correct_predictions = [
    predictions[i] == df.loc[i, 'answer']
    for i in valid_indices
]

accuracy = sum(correct_predictions) / len(correct_predictions) if correct_predictions else 0.0
print(f"\nAccurasy: {accuracy * 100:.2f}% (Number of invalid predictions: {invalid_cnt})")
```

```bash
Accurasy: 37.76% (Number of invalid predictions: 1)
```

## 44. 対話

以下の問いかけに対する応答を生成せよ.

> つばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。東急大井町線の大井町方面の電車に乗り換えたとき、各駅停車に乗車すべきところ、間違えて急行に乗車してしまったことに気付きました。自由が丘の次の急行停車駅で降車し、反対方向の電車で一駅戻った駅がつばめちゃんの目的地でした。目的地の駅の名前を答えてください。

参考: [東急線・みなとみらい線路線案内](https://www.tokyu.co.jp/railway/station/map.html)

```python
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

model_name = "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = LLM(
    model=model_name,
    tensor_parallel_size=1,
    dtype="float16"
)

sampling_params = SamplingParams(
    temperature=0.6, top_p=0.9, max_tokens=512, stop="<|eot_id|>"
)

system_prompt = {
    "role": "system", 
    "content": "あなたは日本の鉄道員です。"
}

user_prompt = "つばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。東急大井町線の大井町方面の電車に乗り換えたとき、各駅停車に乗車すべきところ、間違えて急行に乗車してしまったことに気付きました。自由が丘の次の急行停車駅で降車し、反対方向の電車で一駅戻った駅がつばめちゃんの目的地でした。目的地の駅の名前を答えてください。"

message = [
    system_prompt,
    {
        "role": "user",
        "content": user_prompt
    }
]

prompt = tokenizer.apply_chat_template(
    message, tokenize=False, add_generation_prompt=True
)

outputs = llm.generate(prompt, sampling_params)
result = outputs[0].outputs[0].text.strip()

print("解答:", result)
```

```bash
解答: つばめちゃんの目的地は **緑が丘駅** です。

東急東横線と東急大井町線は、自由が丘駅で接続しています。東急大井町線の急行は、自由が丘駅から次の停車駅である **緑が丘駅** に止まります。
```

## 45. マルチターン対話

先ほどの応答に続けて, 以下の追加の問いかけに対する応答を生成せよ.

> さらに、つばめちゃんが自由が丘駅で乗り換えたとき、先ほどとは反対方向の急行電車に間違って乗車してしまった場合を考えます。目的地の駅に向かうため、自由が丘の次の急行停車駅で降車した後、反対方向の各駅停車に乗車した場合、何駅先の駅で降りれば良いでしょうか？

```Python
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

model_name = "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = LLM(
    model=model_name,
    tensor_parallel_size=1,
    dtype="float16"
)

sampling_params = SamplingParams(
    temperature=0.6, top_p=0.9, max_tokens=512, stop="<|eot_id|>"
)

system_prompt = {
    "role": "system", 
    "content": "あなたは日本の鉄道員です。"
}

user_prompts = [
    "つばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。東急大井町線の大井町方面の電車に乗り換えたとき、各駅停車に乗車すべきところ、間違えて急行に乗車してしまったことに気付きました。自由が丘の次の急行停車駅で降車し、反対方向の電車で一駅戻った駅がつばめちゃんの目的地でした。目的地の駅の名前を答えてください。",
    "さらに、つばめちゃんが自由が丘駅で乗り換えたとき、先ほどとは反対方向の急行電車に間違って乗車してしまった場合を考えます。目的地の駅に向かうため、自由が丘の次の急行停車駅で降車した後、反対方向の各駅停車に乗車した場合、何駅先の駅で降りれば良いでしょうか？"
]

message = [
    system_prompt,
    {
        "role": "user",
        "content": user_prompts[0]
    }
]

prompt = tokenizer.apply_chat_template(
    message, tokenize=False, add_generation_prompt=True
)

outputs1 = llm.generate(prompt, sampling_params)
result1 = outputs1[0].outputs[0].text.strip()

message = [
    system_prompt,
    {
        "role": "user",
        "content": user_prompts[0]
    }
]

message.append({"role": "assistant", "content": result1})
message.append({"role": "user", "content": user_prompts[1]})

prompt = tokenizer.apply_chat_template(
    message, tokenize=False, add_generation_prompt=True
)

outputs2 = llm.generate(prompt, sampling_params)
result2 = outputs2[0].outputs[0].text.strip()

print("解答:", result2)
```

```bash
解答: つばめちゃんは自由が丘駅で乗り換えた後、反対方向の急行に乗車してしまい、自由が丘の次の急行停車駅で降車しました。

つばめちゃんの目的地は緑が丘駅なので、反対方向の各駅停車に乗車して、 **1駅** 先の駅で降りれば良いでしょう。

緑が丘駅は、自由が丘駅から東急大井町線で **1駅** 先にあります。
```

## 46. 川柳の生成

適当なお題を設定し, 川柳の案を10個作成せよ.

```python
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

model_name = "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = LLM(
    model=model_name,
    tensor_parallel_size=1,
    dtype="float16"
)

sampling_params = SamplingParams(
    temperature=0.6, top_p=0.9, max_tokens=512, stop="<|eot_id|>"
)

system_prompt = {
    "role": "system", 
    "content": "あなたは日本の川柳家です。"
}

user_prompt = "「春」をお題として川柳を10句作ってください。"

message = [
    system_prompt,
    {
        "role": "user",
        "content": user_prompt
    }
]

prompt = tokenizer.apply_chat_template(
    message, tokenize=False, add_generation_prompt=True
)

outputs = llm.generate(prompt, sampling_params)
result = outputs[0].outputs[0].text.strip()

print("解答:", result)
```

```bash
解答: ## 春の川柳

1.  桜舞う　風に誘われて　花見行く
2.  鳥のさえずり　朝の光に　春を告げる
3.  雪解け水　田んぼに湧き　命の息吹
4.  芽吹き出す　緑の葉　春の息吹
5.  夕暮れ時　茜色に染まる　春の空
6.  雲一つ　浮かぶ青空　春の始まり
7.  温かい日差し　心も軽く　笑顔が溢れる
8.  雪解け水　小川の流れ　春を運ぶ
9.  花々咲き　世界は彩り　春の魔法
10. 春の息吹　新しい始まり　希望に満ちて
```

## 47. LLMによる評価

大規模言語モデルを評価者 (ジャッジ) として, 問題46の川柳の面白さを10段階で評価せよ.

```python
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

model_name = "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = LLM(
    model=model_name,
    tensor_parallel_size=1,
    dtype="float16"
)

sampling_params = SamplingParams(
    temperature=0.6, top_p=0.9, max_tokens=1024, stop="<|eot_id|>"
)

system_prompt = {
    "role": "system",   
    "content": "あなたは日本の川柳評論家です。以下に、「春」をお題とした川柳を示します。その川柳の面白さを1～10の10段階で評価してください。ただし、許容される出力は「1」から「10」までの整数値のみです。すなわち、出力可能トークン数は1です。"
}

senryu_examples = [
    "桜舞う　風に誘われて　花見行く",
    "鳥のさえずり　朝の光に　春を告げる",
    "雪解け水　田んぼに湧き　命の息吹",
    "芽吹き出す　緑の葉　春の息吹",
    "夕暮れ時　茜色に染まる　春の空",
    "雲一つ　浮かぶ青空　春の始まり",
    "温かい日差し　心も軽く　笑顔が溢れる",
    "雪解け水　小川の流れ　春を運ぶ",
    "花々咲き　世界は彩り　春の魔法",
    "春の息吹　新しい始まり　希望に満ちて"
]

user_prompt = []
scores = []
for senryu in senryu_examples:
    user_prompt = {"role": "user", "content": senryu}
    
    message = [system_prompt, user_prompt]

    prompt = tokenizer.apply_chat_template(
        message, tokenize=False, add_generation_prompt=True
    )

    outputs = llm.generate(prompt, sampling_params)
    result = outputs[0].outputs[0].text.strip()
    
    scores += result
    
for senryu, score in zip(senryu_examples, scores):
    print(f"川柳: {senryu}")
    print(f"評価: {score}")
```

```bash
川柳: 桜舞う　風に誘われて　花見行く
評価: 7
川柳: 鳥のさえずり　朝の光に　春を告げる
評価: 7
川柳: 雪解け水　田んぼに湧き　命の息吹
評価: 8
川柳: 芽吹き出す　緑の葉　春の息吹
評価: 8
川柳: 夕暮れ時　茜色に染まる　春の空
評価: 7
川柳: 雲一つ　浮かぶ青空　春の始まり
評価: 7
川柳: 温かい日差し　心も軽く　笑顔が溢れる
評価: 8
川柳: 雪解け水　小川の流れ　春を運ぶ
評価: 8
川柳: 花々咲き　世界は彩り　春の魔法
評価: 7
川柳: 春の息吹　新しい始まり　希望に満ちて
評価: 7
```

## 48. LLMによる評価の頑健性

問題47で行ったLLMによるテキストの評価に関して, その頑健さ (脆弱さ) を調査せよ. 最も単純な方法は, 同じ評価を何回か繰り返した時のスコアの分散を調べることであろう. また, 川柳の末尾に特定のメッセージを追加することで, 評価スコアを恣意的に操作することも可能であろう.

## 49. トークン化

以下の文章 (夏目漱石の『吾輩は猫である』の冒頭部分) のトークン数を計測せよ.

> 吾輩は猫である。名前はまだ無い。
>
>どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢ったがこんな片輪には一度も出会わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙を吹く。どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。

```python
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

model_name = "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = LLM(
    model=model_name,
    tensor_parallel_size=1,
    dtype="float16"
)

sampling_params = SamplingParams(
    temperature=0.6, top_p=0.9, max_tokens=512, stop="<|eot_id|>"
)

system_prompt = {
    "role": "system",   
    "content": "あなたは日本の文章校正者です。以下に示す文章のトークン数を計測してください。"
}

nekodearu = """吾輩は猫である。名前はまだ無い。

どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢ったがこんな片輪には一度も出会わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙を吹く。どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。"""

message = [
    system_prompt,
    {
        "role": "user",
        "content": nekodearu
    }
]

prompt = tokenizer.apply_chat_template(
    message, tokenize=False, add_generation_prompt=True
)

outputs = llm.generate(prompt, sampling_params)
result = outputs[0].outputs[0].text.strip()

print("解答:", result)
```

```bash
解答: この文章のトークン数は、**324**です。

トークンは、単語や句読点など、文章の要素を表す最小単位です。

この文章では、単語の分割や句読点の扱いによって、トークン数が変化する可能性があります。
```